### **MODELO DE MACHINE LEARNING** ###

#### **SISTEMAS DE RECOMENDACIÓN**

A continuacion desarollaremos un modelo de sistema de recomedacion para peliculas, con nuestro dataset de muestra. con este entrenaremos al modelo para que luego lo podamos deployar a una interfas grafica que se usada por los consumidores. 

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [2]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow.parquet as pq

In [3]:
df = pd.read_parquet(r'/Users/CarlaCarrasco/Desktop/PROYECTOINDIVIDUALUNO/dataset_ML.parquet')
#df=pd.read_parquet(r'/Users/CarlaCarrasco/Desktop/PROYECTOINDIVIDUALUNO/dataset_COMPLETO.parquet')

In [4]:
df.head()

,id,title,userId,score
2812758,as5866,back to gaya,57727,3.5
2182950,as4553,the exorcism in amarillo,55215,5.0
4567909,as9535,pune 52,19690,3.0
257814,as538,the mine adventure with shawn the train and te...,116794,2.0
4250346,as8869,arthur & merlin: knights of camelot,258137,5.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399998 entries, 2812758 to 8662390
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      399998 non-null  object 
 1   title   399998 non-null  object 
 2   userId  399998 non-null  int64  
 3   score   399998 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.3+ MB


CREACIÓN Y ENTRENAMIENTO DEL MODELO.

In [6]:
#Creo una escala de la puntuacion, va desde el 1 al 5. 
reader = Reader(rating_scale=(1, 5))

Creo un objeto Dataset(formato con que trabaja la libreria) de Surprise utilizando la función load_from_df(), convirtiendo mi DataFrame de pandas a un objeto que pueda trabajar.
Selecciono solo 'userId', 'title' y 'score' df **df** y con la función load_from_df(). El orden de las columnas debe ser [usuario, ítem, puntuación], esta se encarga de unir nuestra variable que contenia el score, junto con las demas de df, y convertirlo en un modelo entrenable por la librería.

In [7]:
data = Dataset.load_from_df(df[['userId', 'title', 'score']], reader)

Una vez, el data set armado, lo podemos usar para comenzar con el modelo.

Primero dividimos los parametros para *'train'* y *'test'*

In [8]:
trainset, testset = train_test_split(data, test_size=0.2)

Instancio el modelo.

In [9]:
model = SVD()

Entreno el modelo.

In [10]:
model.fit(trainset)

Testeo el modelo.

In [11]:
predictions = model.test(testset)


**EVALUACION**

Calculo el error cuadrático medio (RMSE) y el error absoluto medio (MAE) utilizando accuracy.rmse() y accuracy.mae() de Surprise.

In [12]:
from surprise import accuracy #traigo las funciones que necesito

accuracy.rmse(predictions)
accuracy.mae(predictions)


RMSE: 1.0319
MAE:  0.8147


0.8146974748259048

Optimización de hiperparámetros.

In [13]:
from surprise.model_selection import cross_validate
import numpy as np

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128,256]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0271  1.0254  1.0251  1.0259  0.0009  
Fit time          1.63    1.70    1.65    1.66    0.03    
Test time         1.22    1.18    0.93    1.11    0.12    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0242  1.0288  1.0250  1.0260  0.0020  
Fit time          1.60    1.70    1.64    1.65    0.04    
Test time         1.18    1.20    0.97    1.12    0.11    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0278  1.0253  1.0251  1.0261  0.0012  
Fit time          1.69    1.71    1.71    1.70    0.01    
Test time         1.00    0.98    1.19    1.05    0.10    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0281  1.0

### **FUNCIÓN DE PREDICCIÓN**

In [14]:
from surprise import dump
import joblib

In [15]:
# Cargar el modelo desde el archivo
file_name = 'modelo_entrenado.sav'
loaded_model = dump.load(file_name)[1]

In [16]:
modelo = joblib.load('modelo_entrenado.sav')

def recomendacion(userId, id):
    
    prediction = model.predict(userId, id)

    if prediction.est >= 3.5:
        
        consejo = "Excelente timing! La película espera por vos."
        
    else:
        
        consejo = " Hey! Deberías ir en busca de otra película, parece que esta no es tu crush cinéfilx."


    return consejo

In [17]:
recomendacion(12342, 'as123')

' Hey! Deberías ir en busca de otra película, parece que esta no es tu crush cinéfilx.'

In [18]:
recomendacion(113452, 'as123')

'Excelente timing! La película espera por vos.'